# Récupère tous les points des bâtiments depuis une API

In [1]:
import geopandas as gpd

centroid_versailles = gpd.read_file("emprise-batie-metropole-du-grand-paris.csv", encoding='latin1', on_bad_lines='skip')
print(centroid_versailles.head())

c:\Users\bapti\anaconda3\lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver CSV does not support open option ON_BAD_LINES
  return ogr_read(


                                Geo Point  \
0  48.688642156684246, 2.3462712918511572   
1     48.6871471110328, 2.355957246360331   
2  48.688148345604866, 2.3303410201219905   
3  48.686297862250576, 2.3607111364661155   
4    48.68708554964789, 2.358796572798365   

                                           Geo Shape objectid    n_sq_eb  \
0  {"coordinates": [[[2.34625533386528, 48.688701...   665501  910026671   
1  {"coordinates": [[[2.35593861413946, 48.687086...   665503  910026673   
2  {"coordinates": [[[2.330302668589308, 48.68812...   665527  910026696   
3  {"coordinates": [[[2.360682459434178, 48.68624...   665534  910026702   
4  {"coordinates": [[[2.358842500507619, 48.68710...   665539  910026706   

  dur_code              h_moy h_med h_min h_max            surface    n_sq_co  \
0       01         6.86197691   6.9   4.5   8.7  90.08079996999999  910001140   
1       01  7.804654289999999   8.0   3.5  10.1        84.00525001  910001140   
2       02         4.22166667

# Récupère les bâtiments qui sont dans la zone des trajectoires

In [2]:
# Supprimer les lignes où la colonne "Geo Point" est vide
centroid_versailles = centroid_versailles[centroid_versailles['Geo Point'].notna() & (centroid_versailles['Geo Point'] != '')]

# Afficher le nombre de lignes restantes
print(f"Nombre de lignes après suppression des valeurs vides : {len(centroid_versailles)}")
# Diviser la colonne "Geo Point" en latitude et longitude
centroid_versailles[['latitude', 'longitude']] = centroid_versailles['Geo Point'].str.split(',', expand=True).astype(float)

# Afficher les premières lignes pour vérifier
print(centroid_versailles[['latitude', 'longitude']].head())
print(centroid_versailles.head())

Nombre de lignes après suppression des valeurs vides : 996108
    latitude  longitude
0  48.688642   2.346271
1  48.687147   2.355957
2  48.688148   2.330341
3  48.686298   2.360711
4  48.687086   2.358797
                                Geo Point  \
0  48.688642156684246, 2.3462712918511572   
1     48.6871471110328, 2.355957246360331   
2  48.688148345604866, 2.3303410201219905   
3  48.686297862250576, 2.3607111364661155   
4    48.68708554964789, 2.358796572798365   

                                           Geo Shape objectid    n_sq_eb  \
0  {"coordinates": [[[2.34625533386528, 48.688701...   665501  910026671   
1  {"coordinates": [[[2.35593861413946, 48.687086...   665503  910026673   
2  {"coordinates": [[[2.330302668589308, 48.68812...   665527  910026696   
3  {"coordinates": [[[2.360682459434178, 48.68624...   665534  910026702   
4  {"coordinates": [[[2.358842500507619, 48.68710...   665539  910026706   

  dur_code              h_moy h_med h_min h_max            surface

In [3]:
# Créer un nouveau dataset avec uniquement latitude et longitude
geo_points_dataset = centroid_versailles[['latitude', 'longitude']]

# Afficher les premières lignes pour vérifier
print(geo_points_dataset.head())
print("Valeurs minimales et maximales de latitude :", centroid_versailles['latitude'].min(), centroid_versailles['latitude'].max())
print("Valeurs minimales et maximales de longitude :", centroid_versailles['longitude'].min(), centroid_versailles['longitude'].max())

    latitude  longitude
0  48.688642   2.346271
1  48.687147   2.355957
2  48.688148   2.330341
3  48.686298   2.360711
4  48.687086   2.358797
Valeurs minimales et maximales de latitude : 48.64642207457913 49.02276328922562
Valeurs minimales et maximales de longitude : 2.145943438278944 2.612682292502828


# Créer un dataframe avec les coordonnées des bâtiments sous forme géospatiale

In [ ]:
# Définir les limites de la bounding box
min_lon, min_lat, max_lon, max_lat = 2.00, 48.7, 2.40, 49.0

# Filtrer les points dans la bounding box
filtered_points = geo_points_dataset[
    (geo_points_dataset['longitude'] >= min_lon) &
    (geo_points_dataset['longitude'] <= max_lon) &
    (geo_points_dataset['latitude'] >= min_lat) &
    (geo_points_dataset['latitude'] <= max_lat)
]

# Afficher les premières lignes filtrées
print(filtered_points[['latitude', 'longitude']].head())

     latitude  longitude
11  48.936585   2.226334
12  48.934806   2.221537
13  48.937153   2.221546
14  48.940815   2.220602
15  48.940284   2.217001


In [1]:
from shapely.geometry import Point
# Créer des points géométriques à partir des colonnes 'longitude' et 'latitude'
geometry = [Point(xy) for xy in zip(filtered_points['longitude'], filtered_points['latitude'])]

gdf = gpd.GeoDataFrame(geometry=geometry, crs="EPSG:4326")
print(gdf.head())

NameError: name 'filtered_points' is not defined

# Enregistrer les bâtiments pour l'utiliser avec le code my-home-is-my-secret

In [ ]:
gdf.to_file("../../my-home-is-my-secret/my-home-is-my-secret-master/input/centroids_versailles.shp")